# HW 2 Multimodal Machine Learning for Emotion Recognition

- main with sub notebooks
    1. audio (acoustic) 
    2. text (lexical) 
    3. visual (this notebook)
    
- `%load` and `%run` with [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-load): This allows us to get code from another notebook

In [7]:
main_notebook = 'main.ipynb'

In [8]:
# %load main_notebook
main.ipynb

In [9]:
%run 'main.ipynb'

- If video, the original video will be a 3D with a T dimension. This specific case, we have a 2D of (T, 2048) which is based on the ResNet encoder

In [ ]:
for visual_features_path in visual_features_paths:
    # print("Current path with files is: ", visual_features_path)
    path_exists = os.path.exists(visual_features_path)
    # print(path_exists)
    if path_exists == True:
        print("Current path with files is: ", visual_features_path)
        load_visual_features_file = np.load(visual_features_path)
        print("  Original Shape: ", np.shape(load_visual_features_file))
        resampled_visual = np.mean(load_visual_features_file, axis=0)
        print("  Reduced shape: ", np.shape(resampled_visual))
        print()
    else:
        print("\nCANNOT find current path: ", visual_features_path)
    

Current path with files is:  /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy
  Original Shape:  (41, 2048)
  Reduced shape:  (2048,)

Current path with files is:  /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_M011.npy
  Original Shape:  (218, 2048)
  Reduced shape:  (2048,)

Current path with files is:  /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F002.npy
  Original Shape:  (172, 2048)
  Reduced shape:  (2048,)

Current path with files is:  /Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F003.npy